In [1]:
import cv2
from keras.models import load_model
import numpy as np
import random
import time

model = load_model('keras_model.h5')
cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype = np.float32)

font = cv2.FONT_HERSHEY_SIMPLEX
colour = (225, 255, 255)

options = {
    0: "Rock",
    1: "Paper",
    2: "Scissors",
    3: "Nothing"
}

def map_of_options(option_choice):
    return options[option_choice]

""" 
Intro Screen: Do you want to play Rock, Paper, Scissors?
 
Playing Screen: Display countdown, choices and score 
1. Countdown from 5
2. At 0s choices displayed on screen & who won that round 
3. Add point to winner's tally
4. Repeat 
5. When player or computer reaches 3 points, end the loop & move to End Screen 

End Screen: Display the winner. q to close window 
"""

def countdown(t):
    """
    Countdown from 5 seconds, at 0 seconds display winner for that round 
    """
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:01d}'.format(secs)
        print("Make your choice in", timer, "seconds")
        time.sleep(1)
        t -=1
    print("Out of time!")


t = 5

def game():
    t = 5
    previous_move = None
    players_points = 0 
    computers_points = 0
    rounds = 5 
    winner = "Waiting"

    while rounds >= 0: 

        ret, frame = cap.read() # ret: return if read (a boolean), frame: will get next frame in camera via "cap" (an image array vector)
        resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
        image_np = np.array(resized_frame)
        normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
        data[0] = normalized_image

        #predict the move made 
        prediction = model.predict(data)
        move_code = np.argmax(prediction[0])
        players_move = map_of_options(move_code)

        if prediction[0][0]>0.7:
            prediction = options[0] # Rock
        elif prediction[0][1]>0.7:
            prediction = options[1] # Paper
        elif prediction[0][2]>0.7:
            prediction = options[2] # Scissors 
        else:
            prediction = options[3] # Nothing 
        
        if previous_move != players_move:
            
            if players_move != "Nothing":
                
                computers_move = random.choice(options)
                #winner_func()

                if computers_move == players_move: 
                    winner = "Draw"
                    rounds -= 1 
                    print(f"players points: {players_points}")
                    print(f"computers points: {computers_points}")

                elif (players_move == "Rock" and computers_move == "Scissors") or (players_move == "Paper" and computers_move == "Rock") or (players_move == "Scissors" and computers_move == "Paper"):
                    players_points += 1
                    rounds -= 1 
                    winner = "Player"
                    print(f"players points: {players_points}")
                    print(f"computers points: {computers_points}")

                elif (computers_move == "Rock" and players_move == "Scissors") or (computers_move == "Paper" and players_move == "Rock") or (computers_move == "Scissors" and players_move== "Paper"):
                    computers_points += 1
                    rounds -= 1 
                    winner = "Computer"
                    print(f"players points: {players_points}")
                    print(f"computers points: {computers_points}")
                    
                        
            else:
                computers_move = "Nothing"
                winner = "Waiting"
        
        previous_move = players_move

        cv2.putText(frame, "Make your choice in: ", (100, 50), font, 1.2, colour, 2)
        cv2.putText(frame, "You have chosen: " + players_move, (100, 100), font, 1.2, colour, 2)
        cv2.putText(frame, f"Your score is: {players_points}", (100, 150), font, 1.2, colour, 2)
        cv2.putText(frame, "Computer's Move: " + computers_move, (1250, 50), font, 1.2, colour, 2)
        cv2.putText(frame, f"Computer's score is: {computers_points}", (1250,100), font, 1.2, colour, 2)
        cv2.putText(frame, f"ROUND: {rounds}     WINNER: {winner}", (500, 1000), font, 2, colour, 4)
        
        if computers_move != "Nothing":
            icon = cv2.imread(f"{computers_move}.png".format(computers_move), 1, )
            icon = cv2.resize(icon, (200, 200))
            frame[150:350, 1400:1600] = icon

        cv2.imshow("Rock Paper Scissors", frame)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
    
        if rounds == 0:
            break

    if (players_points > computers_points):
        cv2.putText(frame, "You win!" , (500, 500), font, 1.5, colour, 4)
        cv2.waitKey(3000)
        cap.release()
        cv2.destroyAllWindows

    elif (computers_points > players_points):
        cv2.putText(frame, "You lose!" , (500, 500), font, 1.5, colour, 4)
        cv2.waitKey(3000)
        cap.release()
        cv2.destroyAllWindows

    else:
        cv2.putText(frame, "You Draw!" , (500, 500), font, 1.5, colour, 4)
        cv2.waitKey(3000)
        cap.release()
        cv2.destroyAllWindows

game() 



2022-03-24 21:19:11.170119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-24 21:19:11.170224: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-03-24 21:19:13.468201: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-24 21:19:13.618709: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


players points: 1
computers points: 0
players points: 1
computers points: 0
players points: 1
computers points: 1
players points: 1
computers points: 1
players points: 2
computers points: 1


In [23]:
import time 

def countdown(t):
    """
    Countdown from 5 seconds, at 0 seconds display winner for that round 
    """
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:01d}'.format(secs)
        print("Make your choice in", timer, "seconds")
        time.sleep(1)
        t -=1
    print("Out of time!")

t = 5

countdown(t)


Make your choice in 5 seconds
Make your choice in 4 seconds
Make your choice in 3 seconds
Make your choice in 2 seconds
Make your choice in 1 seconds
Out of time!
